In [1]:
import os
import cv2
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image


In [2]:

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-printed')


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

c:\Users\oyku_\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\oyku_\.cache\huggingface\hub\models--microsoft--trocr-large-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-printed and are newly initialized: ['enc

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:

def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

    # Find lines in the image
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    # Create a mask for the lines
    line_mask = np.zeros_like(image)

    # Draw detected lines on the mask
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_mask, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Find contours of the lines to extract text regions
    contours, _ = cv2.findContours(cv2.cvtColor(line_mask, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Process each contour
    extracted_texts = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Extract the line of text
        line_image = image[y:y+h, x:x+w]

        # Convert to PIL image for processing with TrOCR
        pil_image = Image.fromarray(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))

        # Process the line with TrOCR
        pixel_values = processor(images=pil_image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Store the extracted text
        extracted_texts.append(generated_text)

    return extracted_texts

def process_images_in_directory(directory):
    all_extracted_texts = {}

    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(directory, filename)
            print(f'Processing {filename}...')
            extracted_texts = process_image(image_path)
            all_extracted_texts[filename] = extracted_texts

    return all_extracted_texts

# Example usage
directory = 'C:\\Users\\oyku_\\Desktop\\image_pro\\recipe_pics\\recipes' 
all_texts = process_images_in_directory(directory)

# Print all extracted texts
for image_name, texts in all_texts.items():
    print(f"\nTexts extracted from {image_name}:")
    for text in texts:
        print(text)


Processing image_2.png...


c:\Users\oyku_\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The channel dimension is ambiguous. Got image shape (3, 603, 3). Assuming channels are the first dimension.


Processing recipe_img (1).jpg...


The channel dimension is ambiguous. Got image shape (3, 338, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 205, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 164, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 377, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 111, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 204, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 123, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 312, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 304, 3). Assuming channels are the first dimension.
The channel dimension is amb

Processing recipe_img (2).jpg...


...

Processing recipe_img_test.jpg...
Processing recipe_img_test_2.jpg...
Processing test.jpg...


The channel dimension is ambiguous. Got image shape (3, 235, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 251, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 228, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 277, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 274, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 118, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 247, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 188, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 114, 3). Assuming channels are the first dimension.


Processing test_hand.jpg...


The channel dimension is ambiguous. Got image shape (3, 137, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 116, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (3, 152, 3). Assuming channels are the first dimension.



Texts extracted from image_2.png:
AMMOVAY
MONROE

Texts extracted from recipe_img (1).jpg:
TOTAL :
@WATEROTWHIMSY
THE USE ARE BEEME REQUIRED OF CROSS OR REQUIRED
ITEM KING TIME INCLUSIVE
TOP AND SPREAD
THANK YOU ARE REVERSE
THANK YOU ARE ARE MORE INCLUSIVE RECEIPT FOR AN OFFER
ITEMLINE:
SATIN AND RETURN MEAT
AVIMHIA IN MISH MADE EINSAVA
YOUR CHILLED STREUSEL OUT OF
7.SLOWLY ADD YOUR DRY INGREDIENTS TO THE WET IN
6. IN A SEPARATE BOWL, MIX ALL THE DRY INGR
AND PUMPKIN PUREE TOGETHER.
4. IN A LARGE BOWL, MIX OIL, SUGAR,
ITEM
INNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
REFERENCE EXCHANGE ARE BE MORE REFERENT
INNNNNNNNNNNNNNN
1.MIX ALL THE STROUSEL INGREDIENTS TOGETHER AND
4
ITEM
GROUND CLOVES
TOTAL
T-
SALES EXCHANGE INCE
TSD. CINNAMON
1/4 CUP BROWN SUG
THANK YOU ARE ARE
ITEM
1/4 CUN MANLE SUMMARY
HHHHHHHHHH
3
ITEM
1/2 CUP POWDERED SUGAR
1/2 CUP VEGETABLE OIL
PARA YAYA
INVIVENTAL
ITEM
TOTAL:
1
.................

Texts extracted from recipe_img (2).jpg:
TOTAL:
RECEIVE BY WILLAN CULSING
SUBJECT:
A